In [1]:
"""將tensorflow檔轉成tensorflow lite，使機器負載量變小"""
from tensorflow import lite
from tensorflow.keras import models

# Parameters
keras_model_filename = './h5/recording1.h5' #訓練好的模型
tflite_filename = './tflite/recording1.tflite' #預建置檔案

# Convert model to TF Lite model
model = models.load_model(keras_model_filename) #載入本來的模型
converter = lite.TFLiteConverter.from_keras_model(model) #將模型載入轉換器
tflite_model = converter.convert() #進行轉換
open(tflite_filename, 'wb').write(tflite_model) #輸出轉換後的模型


INFO:tensorflow:Assets written to: C:\Users\r6017\AppData\Local\Temp\tmp3whvpxj8\assets


188036

In [4]:
import sounddevice as sd
import numpy as np
import scipy.signal
import timeit
import python_speech_features
import tensorflow as tf
from PyQt5.QtCore import QTime
#import RPi.GPIO as GPIO

# Parameters
debug_time = 0 #Debug用
debug_acc = 1 #Debug用
led_pin = 8 #LED PIN
word_threshold = 0.5 #預測值>0.5，表示stop
rec_duration = 0.5 #每一段錄音持續時間
#window_stride = 0.5
sample_rate = 48000 #取樣率(依MIC不同而改變)
resample_rate = 8000 #重整後的取樣率(符合MODEL)
num_channels = 1 #音訊深度
num_mfcc = 16 #回傳mfcc的量
model_path = './tflite/recording1.tflite'
words = ['ㄏㄧㄡ', 'ㄟ', '吼', '啦', '嗯', '的一個', '的這個', '的那個', '空白', '著', '那', '那那個', '阿']#答案對應到的字詞

s = 0 #秒
m = 0 #分
h = 0 #時

# Sliding window
window = np.zeros(int(rec_duration * resample_rate) * 2)#取樣音頻數據變數

# GPIO 
#GPIO.setwarnings(False)
#GPIO.setmode(GPIO.BOARD)
#GPIO.setup(8, GPIO.OUT, initial=GPIO.LOW)

# Load model (interpreter)
interpreter = tf.lite.Interpreter(model_path)
interpreter.allocate_tensors()
input_details = interpreter.get_input_details()
output_details = interpreter.get_output_details()

#開始計時
counter = QTime()
counter.restart()

# Decimate (filter and downsample)
def decimate(signal, old_fs, new_fs):
    
    #檢查是否降低音頻
    if new_fs > old_fs:
        print("Error: target sample rate higher than original")
        return signal, old_fs
    
    #檢查是否為整數(只能在整數下執行)
    dec_factor = old_fs / new_fs
    if not dec_factor.is_integer():
        print("Error: can only decimate by integer factor")
        return signal, old_fs

    # Do decimation
    resampled_signal = scipy.signal.decimate(signal, int(dec_factor))

    return resampled_signal, new_fs

# This gets called every 0.5 seconds
def sd_callback(rec, frames, time, status):

    #GPIO.output(led_pin, GPIO.LOW)
    
    # Notify if errors
    if status:
        print('Error:', status)
    
    # Remove 2nd dimension from recording sample
    #壓縮成1D張量
    rec = np.squeeze(rec)
    
    # Resample
    #重取樣成8000HZ(以符合訓練模型)
    rec, new_fs = decimate(rec, sample_rate, resample_rate)
    
    # Save recording onto sliding window
    #將音訊輸入到window
    window[:len(window)//2] = window[len(window)//2:]
    window[len(window)//2:] = rec

    # Compute features
    mfccs = python_speech_features.base.mfcc(window, #輸入訊號
                                        samplerate=new_fs, #取樣率
                                        winlen=0.256, #音框涵蓋時間
                                        winstep=0.050, #音框間距離
                                        numcep=num_mfcc, #返回係數的量
                                        nfilt=26, #過濾器數量
                                        nfft=2048,#FFT大小
                                        preemph=0.0,#不用預強化濾波器
                                        ceplifter=0,#ROBUST
                                        appendEnergy=False,#係數0的話對被替代成總音框能量的對數
                                        winfunc=np.hanning)#hanning window
    mfccs = mfccs.transpose()

    # Make prediction from model
    in_tensor = np.float32(mfccs.reshape(1, mfccs.shape[0], mfccs.shape[1], 1))
    #設定輸入張量
    interpreter.set_tensor(input_details[0]['index'], in_tensor)
    #進行預測
    interpreter.invoke()
    #取得輸出張量
    output_data = interpreter.get_tensor(output_details[0]['index'])
    
    val = output_data[0]#取得預測值
    val = val.tolist() #np.ndarray to list
    list_val_max = max(val) #取得最大值
    list_val_maxIndex = val.index(max(val)) #取得最大值的索引  
    
    if(list_val_max >= 0.3):#如果預測值>=0.3
        #if(list_val_maxIndex != 8):
            print(words[list_val_maxIndex])#輸出相對應的字詞
            print("MAX:" + str(list_val_max))#輸出預測值當中最大的值
            print(str(h) + "時" + str(m) + "分" + "{:.1f}秒".format(s))
           
    #if debug_acc:
    #    print("pred:" + str(val))#輸出所有預測值
    #    print("MAX:" + str(list_val_max))#輸出預測值當中最大的值
    #    print("MAX_INDEX:" + str(list_val_maxIndex))#輸出最大值的索引值

# Start streaming from microphone
with sd.InputStream(channels=num_channels,
                    samplerate=sample_rate,
                    blocksize=int(sample_rate * rec_duration),
                    callback=sd_callback):
    while True:
        s = float(counter.elapsed() / 1000)
        if(s >= 60):
            counter.restart()
            s = 0
            m = m + 1
        if(m == 60):
            m = 0
            h = h + 1
        pass


空白
MAX:0.9999923706054688
0時0分0.8秒
空白
MAX:0.687397837638855
0時0分1.4秒
著
MAX:0.8309696912765503
0時0分1.8秒
著
MAX:0.5470786690711975
0時0分2.4秒
著
MAX:0.7947686314582825
0時0分2.8秒
著
MAX:0.33827924728393555
0時0分3.4秒
著
MAX:0.7079254388809204
0時0分3.9秒
著
MAX:0.37011706829071045
0時0分4.3秒
著
MAX:0.3348577320575714
0時0分5.8秒
空白
MAX:0.3385903537273407
0時0分6.3秒
阿
MAX:0.3440547585487366
0時0分7.0秒
空白
MAX:0.987105131149292
0時0分7.3秒
空白
MAX:0.7356818318367004
0時0分7.8秒
空白
MAX:0.9871871471405029
0時0分8.3秒
阿
MAX:0.5091067552566528
0時0分8.8秒
阿
MAX:0.8000456094741821
0時0分9.4秒
空白
MAX:0.6496671438217163
0時0分9.9秒
空白
MAX:0.9775400161743164
0時0分10.3秒
嗯
MAX:0.5684104561805725
0時0分10.9秒
著
MAX:0.8330252170562744
0時0分11.3秒
著
MAX:0.6087129712104797
0時0分11.8秒
空白
MAX:0.9544726610183716
0時0分12.3秒
著
MAX:0.7341915369033813
0時0分13.0秒
著
MAX:0.8821176290512085
0時0分13.3秒
著
MAX:0.591705858707428
0時0分13.9秒
空白
MAX:0.8593977689743042
0時0分14.4秒
著
MAX:0.8312952518463135
0時0分14.8秒
阿
MAX:0.8169546723365784
0時0分15.4秒
著
MAX:0.9837095737457275
0時0

KeyboardInterrupt: 

In [15]:
from PyQt5.QtCore import QTime

ms = 0
s = 0
m = 0
h = 0
display = False

counter = QTime()
counter.restart()

while True:
        ms = counter.elapsed()
        if(ms >= 1000):
            counter.restart()
            s = s + 1
            display = True
        if(s == 60):           
            s = 0
            m = m + 1
        if(m == 60):
            m = 0
            h = h + 1
        if(display):
            print(str(h) + "時" + str(m) + "分" + str(s) + "秒" + str(ms) + "毫秒")
            display = False
        pass
   



0時0分1秒1000毫秒
0時0分2秒1000毫秒
0時0分3秒1000毫秒
0時0分4秒1000毫秒
0時0分5秒1000毫秒
0時0分6秒1000毫秒
0時0分7秒1000毫秒
0時0分8秒1000毫秒
0時0分9秒1000毫秒
0時0分10秒1000毫秒
0時0分11秒1000毫秒
0時0分12秒1000毫秒
0時0分13秒1000毫秒
0時0分14秒1000毫秒
0時0分15秒1000毫秒
0時0分16秒1000毫秒
0時0分17秒1000毫秒
0時0分18秒1000毫秒
0時0分19秒1000毫秒
0時0分20秒1000毫秒
0時0分21秒1000毫秒
0時0分22秒1000毫秒
0時0分23秒1000毫秒
0時0分24秒1000毫秒
0時0分25秒1000毫秒
0時0分26秒1000毫秒
0時0分27秒1000毫秒
0時0分28秒1000毫秒
0時0分29秒1000毫秒
0時0分30秒1000毫秒
0時0分31秒1000毫秒
0時0分32秒1000毫秒
0時0分33秒1000毫秒
0時0分34秒1000毫秒
0時0分35秒1000毫秒
0時0分36秒1000毫秒
0時0分37秒1000毫秒
0時0分38秒1000毫秒
0時0分39秒1000毫秒
0時0分40秒1000毫秒
0時0分41秒1000毫秒
0時0分42秒1000毫秒
0時0分43秒1000毫秒
0時0分44秒1000毫秒
0時0分45秒1000毫秒
0時0分46秒1000毫秒
0時0分47秒1000毫秒
0時0分48秒1000毫秒
0時0分49秒1000毫秒
0時0分50秒1000毫秒
0時0分51秒1000毫秒
0時0分52秒1000毫秒
0時0分53秒1000毫秒
0時0分54秒1000毫秒
0時0分55秒1000毫秒
0時0分56秒1000毫秒
0時0分57秒1000毫秒
0時0分58秒1000毫秒
0時0分59秒1000毫秒
0時1分0秒1000毫秒
0時1分1秒1000毫秒
0時1分2秒1000毫秒
0時1分3秒1000毫秒
0時1分4秒1000毫秒
0時1分5秒1000毫秒
0時1分6秒1000毫秒


KeyboardInterrupt: 